In [ ]:
'''
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
'''
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'


  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=89afd4aa6d49dd8bf48d7cafdc48cbfcff64eb94e21fe40461d7cf52c2e6483c
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


Import



In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
import random

from google.colab import drive
drive.mount('/content/drive')
  
%cd /content/drive/My\ Drive/MLDLProjects/Lande_Napolitano_Pipoli/Project_root
 
import import_ipynb
from cifar100 import cifar_100
import net
from net import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/MLDLProjects/Lande_Napolitano_Pipoli/Project_root
importing Jupyter notebook from cifar100.ipynb
IMPORT CIFAR DONE
importing Jupyter notebook from net.ipynb
IMPORT NET DONE


HyperParameters

In [ ]:
LR=2.0
GAMMA=0.2
WEIGHT_DECAY=1e-5
MILESTONE=[49,64]
BATCH_SIZE=128
NUM_EPOCHS=70
TOTAL_CLASSES = 100
NUM_CLASSES = 10
K=2000
MOMENTUM=0.9
DEVICE="cuda"
os.environ['PYTHONWARNINGS'] = "ignore"

iCarl Model

In [ ]:
class iCaRLNet(nn.Module):
    def __init__(self, feature_size, n_classes, lr=LR, gamma=GAMMA, weight_decay=WEIGHT_DECAY, milestone=MILESTONE, momentum=MOMENTUM, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS):
        # Network architecture
        super(iCaRLNet, self).__init__()
        self.net = resnet32(num_classes=n_classes)
        self.feature_extractor = self.net.get_fm_out
        self.lr = lr
        self.gamma = gamma
        self.weight_decay = weight_decay
        self.milestone = milestone
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.n_classes = 0
        self.n_known = 0
        self.feature_size=feature_size
        self.momentum=momentum

        self.loss = nn.BCEWithLogitsLoss()

        self.compute_means = True
        self.exemplar_means = []
        self.exemplar_sets = []
        self.mapper = None


    def forward(self, x):

        self.net = self.net.cuda()
        return self.net.forward(x)


    def increment_classes(self, n):

        in_features = self.fc.in_features
        out_features = self.fc.out_features
        weight = self.fc.weight.data
        self.fc = nn.Linear(in_features, out_features + n, bias=False)
        self.fc.weight.data[:out_features] = weight
        self.n_classes += n


    def classify(self, x, cif, rl): 
        
        if self.compute_means:
            exemplar_means = []
            for i,P_y in enumerate(self.exemplar_sets):  #P_y list of indice
                
                features = np.zeros((0,self.feature_size))
                if i<self.n_classes-NUM_CLASSES:
                    #print("VECCHIE CLASSI")
                    sub = Subset(cif, P_y)
                    #print(len(sub))
                    dl = torch.utils.data.DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 
                    with torch.no_grad():
                        for ind, ex, lab in dl:# Extract feature for each exemplar in P_y
                            
                            ex = Variable(ex).cuda()
                            feature = self.feature_extractor(ex).data.cpu().numpy()
                            feature = normalize(feature, axis=1, norm='l2')
                            features = np.concatenate((features,feature), axis=0)
                else:
                    #print("NUOVE CLASSI")
                    indexes=cif.getClassIndexes(rl[i])
                    sub = Subset(cif, indexes)
                    #print(len(sub))
                    dl = torch.utils.data.DataLoader(sub, batch_size=self.batch_size,shuffle=False, num_workers=4) 
                    with torch.no_grad():
                        for ind, ex, lab in dl:# Extract feature for each exemplar in P_y
                            
                            ex = Variable(ex).cuda()
                            feature = self.feature_extractor(ex).data.cpu().numpy()
                            feature = normalize(feature, axis=1, norm='l2')
                            features = np.concatenate((features,feature), axis=0)


                features = torch.tensor(features)
                mu_y = features.mean(0).squeeze()
                mu_y.data = mu_y.data / torch.norm(mu_y, p=2)  # L2 Normalize
                exemplar_means.append(mu_y)
                
            self.exemplar_means = exemplar_means
            self.compute_means = False

        exemplar_means = self.exemplar_means
        means = torch.stack(exemplar_means)  # (n_classes, feature_size)
        means = torch.stack([means] * self.batch_size)  # (batch_size, n_classes, feature_size)
        means = means.transpose(1, 2)  # (batch_size, feature_size, n_classes)

        feature = self.feature_extractor(x)  # (batch_size, feature_size)
        for i in range(feature.size(0)):  # Normalize
            feature.data[i] = feature.data[i] / torch.norm(feature.data[i], p=2)
        feature = feature.unsqueeze(2)  # (batch_size, feature_size, 1)
        feature = feature.expand_as(means)  # (batch_size, feature_size, n_classes)
        feature = feature.cuda()
        means = means.cuda()

        dists = torch.sqrt((feature - means).pow(2).sum(1)).squeeze()  # (batch_size, n_classes)
        _, preds = dists.min(1) #prev

        return preds


    def construct_exemplar_set(self, images, m):
        
        features = np.zeros((0,self.feature_size))
        indices = np.zeros((0), dtype=int)
        dl = torch.utils.data.DataLoader(images, batch_size=self.batch_size,shuffle=False, num_workers=4)
        with torch.no_grad():
          for ind, img, lab in dl:
            x = Variable(img).cuda()
            feature = self.feature_extractor(x).data.cpu().numpy()
            feature = normalize(feature, axis=1, norm='l2')
            features = np.concatenate((features,feature), axis=0)
            indices = np.concatenate((indices,ind), axis=0)

        class_mean = np.mean(features, axis=0)
        class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

        exemplar_set = []
        exemplar_features = np.zeros((0,64))

        for k in range(1, int(m)+1):
            S = np.sum(exemplar_features, axis=0)
            phi = features
            mu = class_mean
            mu_p = 1.0 / k * (phi + S)
            mu_p = normalize(mu_p, axis=1, norm='l2')
            i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
            exemplar_set.append(indices[i])
            addfeature =  np.expand_dims(features[i], axis=0)
            exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

            #remove duplicates
            features = np.delete(features, i, 0)
            indices = np.delete(indices, i, 0)
            
        self.exemplar_sets.append(exemplar_set)
        

    def reduce_exemplar_sets(self, m):

        for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]


    def combine_dataset_with_exemplars(self, cifar):

        newindexes = []
        for y, P_y in enumerate(self.exemplar_sets):
            exemplar_images = P_y
            exemplar_labels = [y] * len(P_y)
            print(exemplar_images[0].shape)
            newindexes+=cifar.data_append(exemplar_images, exemplar_labels)
        return newindexes


    def exemplarIndexes(self):

        Indexes = []
        for P_y in self.exemplar_sets:
            Indexes += P_y
        return Indexes


    def update_representation(self, cifar, batchindexes, diz):

        prev_model = copy.deepcopy(self)
        prev_model = prev_model.eval().cuda()
        self.compute_means = True

        self.n_classes += 10

        # Form combined training set
        newindexes = []
        if self.n_classes > 10:
            newindexes = self.exemplarIndexes() #IMPORTANT!!!!
        newindexes += list(batchindexes)
        
        reprdata = Subset(cifar, newindexes)

        loader = torch.utils.data.DataLoader(reprdata, batch_size=self.batch_size,shuffle=True, num_workers=4, drop_last=True)

        optimizer = optim.SGD(self.net.parameters(), lr=self.lr, momentum=self.momentum,weight_decay=self.weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestone, gamma=self.gamma)

        for epoch in tqdm(range(self.num_epochs)):
            losses = []
            for indices, images, labels in loader:
                labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
                images = Variable(torch.FloatTensor(images)).cuda()
                labels = Variable(labels).cuda()
                optimizer.zero_grad()
                g = self.forward(images)
                
                y_hot = F.one_hot(labels, self.n_classes).float().cuda()
                
                if self.n_known > 0:
                    q = prev_model.forward(images)
                    q = torch.sigmoid(q)
                    target = torch.cat((q[:,:self.n_known], y_hot[:,self.n_known:self.n_classes]), dim=1)
                    loss = self.loss(g[:,:self.n_classes], target)
                else:
                    loss = self.loss(g[:,:self.n_classes], y_hot[:,:self.n_classes])

                losses.append(loss.item())
                loss.backward()
                optimizer.step()
            
            scheduler.step()
    
    def set_mapper(self, mapper):
        self.mapper = mapper

    def get_mapper(self):
        if self.mapper is None:
            print("ERROR: mapper has not been set. Use model.set_mapper(mapper) first.")
            return None
        return self.mapper

    def compute_confusion_matrix(self, cif, cif_test, rl,plot=False):
      
        #cifar_test = cifar_100(100, 'test')
        dl = torch.utils.data.DataLoader(cif_test, batch_size=128,shuffle=True, num_workers=4, drop_last=True)
        y_true = []
        y_pred = []
        for _, images, labels in dl:
            images = Variable(images).cuda()
            y_true.extend(labels)
            y_pred_tmp = self.classify(images, cif, rl)
            y_pred_tmp = [p.item() for p in y_pred_tmp]
            y_pred.extend(y_pred_tmp)
        
        y_true = [self.mapper[l.item()] for l in y_true]
        acc_matrix = s_cm(y_true, y_pred)

        if plot:
            plt.figure(figsize=(11,11))
            df_cm = pd.DataFrame(acc_matrix)
            sn.heatmap(df_cm, square=True, xticklabels=20, yticklabels=20)
            plt.title("iCaRL confusion matrix")
            plt.show()

        return acc_matrix



Main

In [ ]:
torch.cuda.current_device()
torch.cuda._initialized = True
Flag=True

icarl = iCaRLNet(64, TOTAL_CLASSES)
icarl = icarl.to(DEVICE)

acc_vect = []

cifarTrain = cifar_100(NUM_CLASSES, 'train', 653)
cifarTest = cifar_100(NUM_CLASSES, 'test', 653)
randomlist = cifarTrain.get_classes_list()
diz = cifarTrain.get_dictionary()

print(diz.keys())

icarl.set_mapper(diz)

def compute_accuracy(dl, diz, icarl, rl):
    total = 0.0
    correct = 0.0
    for indices, images, labels in dl:
        labels = torch.tensor([torch.tensor(diz[c.item()]) for c in labels])
        images = Variable(images).cuda()
        preds = icarl.classify(images, cifarTrain, rl)
        total = total + len(labels)
        correct += (preds.data.cpu() == labels).sum()
    acc=100 * correct / total
    return acc


for s in range(10):

    print("\n")
    print('-' * 75)
    print(f"ITERATION: {(s+1)*10}/{100}")

    print("Loading training examples for classes", randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batchindexes = cifarTrain.getClassIndexes(randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES])
    batch = Subset(cifarTrain, batchindexes)

    testindexes = cifarTest.getClassIndexes(randomlist[0:s*NUM_CLASSES + NUM_CLASSES])
    test_set = Subset(cifarTest, testindexes)

    print("Batch size train: {} - Batch size test: {}".format(len(batch), len(test_set)))
        

    train_loader = torch.utils.data.DataLoader(batch, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE,shuffle=True, num_workers=4, drop_last=True) #FALSE

    icarl.train()
    icarl.update_representation(cifarTrain, batchindexes, diz)
    icarl.eval()
    m = K / icarl.n_classes

    icarl.reduce_exemplar_sets(m)

    # Construct exemplar sets for new classes
    for y in randomlist[s*NUM_CLASSES:s*NUM_CLASSES + NUM_CLASSES]:
        imagesInd = cifarTrain.getClassIndexes([y])
        images = Subset(cifarTrain, imagesInd)
        icarl.construct_exemplar_set(images, m)

    icarl.n_known = icarl.n_classes

    acc= compute_accuracy(train_loader, diz, icarl, randomlist)
    print('Train Accuracy: {:.2f}'.format(acc).rstrip())
        
    acc= compute_accuracy(test_loader, diz, icarl, randomlist)
    print('Test Accuracy: {:.2f}'.format(acc).rstrip())
    acc_vect.append(acc.item())

    print('-' * 75)
        
print(acc_vect)

cm = icarl.compute_confusion_matrix(cifarTrain, cifarTest, randomlist, plot=True)

Files already downloaded and verified
Files already downloaded and verified
dict_keys([36, 61, 49, 58, 92, 90, 68, 32, 28, 52, 47, 87, 1, 41, 93, 6, 88, 12, 38, 91, 81, 33, 8, 48, 60, 27, 50, 17, 56, 97, 34, 42, 84, 66, 62, 26, 29, 51, 3, 72, 39, 9, 37, 85, 13, 25, 11, 67, 99, 74, 30, 2, 64, 71, 19, 35, 31, 63, 54, 15, 43, 73, 40, 55, 7, 78, 14, 10, 70, 44, 0, 86, 79, 57, 75, 46, 83, 82, 22, 4, 45, 18, 89, 5, 59, 21, 95, 96, 69, 16, 98, 23, 80, 65, 76, 77, 20, 24, 94, 53])


---------------------------------------------------------------------------
ITERATION: 10/100
Loading training examples for classes [36, 61, 49, 58, 92, 90, 68, 32, 28, 52]
Batch size train: 5000 - Batch size test: 1000



Train Accuracy: 98.48
Test Accuracy: 89.84
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 20/100
Loading training examples for classes [47, 87, 1, 41, 93, 6, 88, 12, 38, 91]
Batch size train: 5000 - Batch size test: 2000



Train Accuracy: 91.55
Test Accuracy: 79.95
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 30/100
Loading training examples for classes [81, 33, 8, 48, 60, 27, 50, 17, 56, 97]
Batch size train: 5000 - Batch size test: 3000



Train Accuracy: 91.47
Test Accuracy: 73.57
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 40/100
Loading training examples for classes [34, 42, 84, 66, 62, 26, 29, 51, 3, 72]
Batch size train: 5000 - Batch size test: 4000



Train Accuracy: 87.04
Test Accuracy: 65.68
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 50/100
Loading training examples for classes [39, 9, 37, 85, 13, 25, 11, 67, 99, 74]
Batch size train: 5000 - Batch size test: 5000


...


Train Accuracy: 86.44
Test Accuracy: 57.12
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 70/100
Loading training examples for classes [43, 73, 40, 55, 7, 78, 14, 10, 70, 44]
Batch size train: 5000 - Batch size test: 7000



Train Accuracy: 80.03
Test Accuracy: 51.52
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 80/100
Loading training examples for classes [0, 86, 79, 57, 75, 46, 83, 82, 22, 4]
Batch size train: 5000 - Batch size test: 8000



Train Accuracy: 81.19
Test Accuracy: 49.52
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 90/100
Loading training examples for classes [45, 18, 89, 5, 59, 21, 95, 96, 69, 16]
Batch size train: 5000 - Batch size test: 9000



Train Accuracy: 78.23
Test Accuracy: 47.82
---------------------------------------------------------------------------


---------------------------------------------------------------------------
ITERATION: 100/100
Loading training examples for classes [98, 23, 80, 65, 76, 77, 20, 24, 94, 53]
Batch size train: 5000 - Batch size test: 10000



Train Accuracy: 83.91
Test Accuracy: 45.89
---------------------------------------------------------------------------
[tensor(89.8438), tensor(79.9479), tensor(73.5734), tensor(65.6754), tensor(60.3566), tensor(57.1162), tensor(51.5191), tensor(49.5212), tensor(47.8237), tensor(45.8934)]
